# Recovering $N_{i,I}^\pm$ from its tessellation

This notebook provides a proof for the proposition

***Proposition***. Given a manifold $N_{i,I}^\pm$ with its natural tessellation by copies of $P$, we can uniquely recover:
- the Long manifold $Z_i$, up to tessellation-preserving isometry;
- the $\Lambda_i$-orbit of $I$;
- the type $\pm$.

Recall that the group of tessellation-preserving isometries of $Y_i$ is $\Pi_i \coloneqq N_G(K_i) / K_i^+$. \
Up to conjugacy, there is a unique order-$17$ subgroup of $\Pi_i$, generated by $\psi = (abcde)^2 \cdot K_i^+$. The group $\Lambda_i$ is defined as $N_{\Pi_i}(\langle \psi \rangle)$.

In [1]:
import itertools
from collections import defaultdict

In [2]:
def multiset(it):
    d = defaultdict(int)
    for k in it:
        d[k] += 1
    return dict(d)

def multimap(it):
    d = defaultdict(set)
    for k,v in it:
        d[k].add(v)
    return dict(d)

> _***Note:***_ The following cell may issue a warning, which can be ignored. _***Note:***_ The following cell may issue a warning, which can be ignored.

In [4]:
qr17 = codes.QuadraticResidueCode(17,GF(2))

In [5]:
A = qr17.generator_matrix(); A

[1 1 1 0 1 0 1 1 1 0 0 0 0 0 0 0 0]
[0 1 1 1 0 1 0 1 1 1 0 0 0 0 0 0 0]
[0 0 1 1 1 0 1 0 1 1 1 0 0 0 0 0 0]
[0 0 0 1 1 1 0 1 0 1 1 1 0 0 0 0 0]
[0 0 0 0 1 1 1 0 1 0 1 1 1 0 0 0 0]
[0 0 0 0 0 1 1 1 0 1 0 1 1 1 0 0 0]
[0 0 0 0 0 0 1 1 1 0 1 0 1 1 1 0 0]
[0 0 0 0 0 0 0 1 1 1 0 1 0 1 1 1 0]
[0 0 0 0 0 0 0 0 1 1 1 0 1 0 1 1 1]

The columns of this matrix $A$ are the 17 vector-valued colors assigned to the facets of $Y_i$.

Next, we define a matrix $R$ such that $R \cdot A_k = A_{k+1}$, where $(A_k)_{k\in \mathbb Z_{17}}$ are cyclically indexed columns of $A$.

In [6]:
R = A[:,1:10] * A[:,:9]^-1
print(R)
print()
print(R * A)

[1 0 1 0 0 1 0 1 1]
[1 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 1 0]

[1 1 0 1 0 1 1 1 0 0 0 0 0 0 0 0 1]
[1 1 1 0 1 0 1 1 1 0 0 0 0 0 0 0 0]
[0 1 1 1 0 1 0 1 1 1 0 0 0 0 0 0 0]
[0 0 1 1 1 0 1 0 1 1 1 0 0 0 0 0 0]
[0 0 0 1 1 1 0 1 0 1 1 1 0 0 0 0 0]
[0 0 0 0 1 1 1 0 1 0 1 1 1 0 0 0 0]
[0 0 0 0 0 1 1 1 0 1 0 1 1 1 0 0 0]
[0 0 0 0 0 0 1 1 1 0 1 0 1 1 1 0 0]
[0 0 0 0 0 0 0 1 1 1 0 1 0 1 1 1 0]


The matrix $R$ has two fixed points: the vectors $(0,0,0,0,0,0,0,0,0)$ and $\omega \coloneqq (1,1,1,1,1,1,1,1,1)$.

In [7]:
(R-1).right_kernel()

Vector space of degree 9 and dimension 1 over Finite Field of size 2
Basis matrix:
[1 1 1 1 1 1 1 1 1]

In [8]:
omega = (R-1).right_kernel().gen(0)

Given the tessellated manifold $N_{i,I}^\pm$, we can single out the facets corresponding to the Coxeter generator $g$
(since they are combinatorial simplices, but have a different volume from the facets $f$). \
They lie on several totally geodesic hypersurfaces, which lift to $2^9$ copies of $X_i$ in the $17$-fold cover $\widehat{M}_{i,I}$.

The action of $R$ on $\mathbb Z_2^9$ has $32$ orbits: $2$ of size $1$ (corresponding to $0$ and $\omega$) and $30$ of size $17$. Consequently, the $2^9$ copies of $X_i$ get sent to $2$ copies of $Z_i$ and $30$ copies of $X_i$, which are the hypersurfaces mentioned above. We can then single out the two copies of $Z_i$, since their volume is smaller.

It remains to recover the $\Lambda_i$-orbit of $I$, and the type in $\{+, -\}$.

We now compute representatives of $R$-orbits of $\mathbb Z_2^9$.

In [9]:
def binary_to_int(v):
    """Converts a binary vector to an integer number. Leftmost bit is least significant."""
    return v.change_ring(ZZ) * vector([2^j for j in range(9)])

In [10]:
def int_to_binary(b):
    """Converts an integer number to a binary vector. Leftmost bit is least significant."""
    return vector(GF(2), [{'0':0,'1':1}[c] for c in bin(b)[:1:-1].ljust(9,'0')], immutable=True)

In [11]:
def representative(v):
    """Returns the smallest binary vector in the R-orbit of v."""
    return int_to_binary(min(binary_to_int(R^j * v) for j in range(17)))

In [12]:
representatives = multiset(representative(v) for v in GF(2)^9)
print(len(representatives))
print()
for k, v in representatives.items():
    print(f"{k}: {v}")

32

(0, 0, 0, 0, 0, 0, 0, 0, 0): 1
(1, 0, 0, 0, 0, 0, 0, 0, 0): 17
(0, 1, 0, 0, 0, 0, 0, 0, 0): 17
(1, 0, 1, 0, 0, 0, 0, 0, 0): 17
(0, 1, 1, 0, 0, 0, 0, 0, 0): 17
(0, 0, 0, 1, 0, 0, 0, 0, 0): 17
(1, 1, 0, 1, 0, 0, 0, 0, 0): 17
(1, 1, 1, 1, 0, 0, 0, 0, 0): 17
(1, 0, 0, 0, 1, 0, 0, 0, 0): 17
(0, 1, 0, 0, 1, 0, 0, 0, 0): 17
(1, 0, 1, 0, 1, 0, 0, 0, 0): 17
(0, 1, 1, 0, 1, 0, 0, 0, 0): 17
(1, 1, 0, 1, 1, 0, 0, 0, 0): 17
(0, 0, 1, 1, 1, 0, 0, 0, 0): 17
(0, 1, 0, 0, 0, 1, 0, 0, 0): 17
(1, 0, 1, 0, 0, 1, 0, 0, 0): 17
(0, 1, 1, 0, 0, 1, 0, 0, 0): 17
(0, 0, 0, 1, 0, 1, 0, 0, 0): 17
(1, 1, 1, 1, 0, 1, 0, 0, 0): 17
(0, 1, 0, 0, 1, 1, 0, 0, 0): 17
(1, 0, 1, 0, 1, 1, 0, 0, 0): 17
(1, 1, 0, 1, 1, 1, 0, 0, 0): 17
(1, 0, 0, 1, 0, 0, 1, 0, 0): 17
(0, 1, 1, 1, 0, 0, 1, 0, 0): 17
(0, 0, 1, 0, 1, 0, 1, 0, 0): 17
(1, 1, 1, 0, 1, 0, 1, 0, 0): 17
(1, 0, 0, 1, 1, 0, 1, 0, 0): 17
(1, 0, 1, 1, 1, 0, 1, 0, 0): 17
(1, 1, 1, 0, 0, 1, 1, 0, 0): 17
(0, 1, 0, 1, 1, 1, 1, 0, 0): 17
(0, 1, 1, 0, 1, 1, 0, 1, 0): 17
(1, 1

Define a labeled graph $\mathcal H$ as follows. Vertices are the size-$17$ orbits of the action of $R$ on $\mathbb Z_2^9$; two orbits $V,W$ are joined by an edge labeled by
$$
    e(V, W) \coloneqq \frac{1}{17} \cdot \left|\{(v,w,j) \mid v \in V, w \in W, j \in \mathbb Z_{17}, v+A_j = w\}\right|
$$
or by no edge if $e(V, W) = 0$. If $v_0$ represents $V$, then we can compute
$$
    e(V, W) = \left|\{(w,j) \mid w \in W, j \in \mathbb Z_{17}, v_0+A_j = w\}\right|.
$$
We will use the notation $[v]$ to refer to the $R$-orbit of a binary vector $v \in \mathbb Z_2^9$.

In [13]:
%%time
verts = [v for v in representatives if representatives[v] == 17]
edges = []
for v, w in itertools.combinations(verts, 2):
    wt = sum(1 for k in range(17) for col in A.columns() if v + col == R^k * w)
    if wt:
        edges.append((v,w,wt))
H = Graph(edges)

CPU times: user 725 ms, sys: 105 μs, total: 725 ms
Wall time: 726 ms


The graph $\mathcal H$ has $30$ vertices and $134$ edges: $14$ labeled $1$ and $120$ labeled $2$.

In [14]:
multiset(H.edge_labels())

{2: 120, 1: 14}

We may characterize vertices by the sum of labels of their incident edges:

In [15]:
sum_edge_labels = multimap((sum(c for a,b,c in H.edges(v)), v) for v in H)
sum_edge_labels

{16: {(1, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 1, 0, 0, 0)},
 17: {(0, 1, 0, 0, 0, 0, 0, 0, 0),
  (1, 0, 1, 0, 0, 0, 0, 0, 0),
  (0, 1, 1, 0, 0, 0, 0, 0, 0),
  (0, 0, 0, 1, 0, 0, 0, 0, 0),
  (1, 1, 0, 1, 0, 0, 0, 0, 0),
  (1, 1, 1, 1, 0, 0, 0, 0, 0),
  (1, 0, 0, 0, 1, 0, 0, 0, 0),
  (0, 1, 0, 0, 1, 0, 0, 0, 0),
  (1, 0, 1, 0, 1, 0, 0, 0, 0),
  (0, 1, 1, 0, 1, 0, 0, 0, 0),
  (1, 1, 0, 1, 1, 0, 0, 0, 0),
  (0, 0, 1, 1, 1, 0, 0, 0, 0),
  (0, 1, 0, 0, 0, 1, 0, 0, 0),
  (1, 0, 1, 0, 0, 1, 0, 0, 0),
  (0, 1, 1, 0, 0, 1, 0, 0, 0),
  (1, 1, 1, 1, 0, 1, 0, 0, 0),
  (0, 1, 0, 0, 1, 1, 0, 0, 0),
  (1, 0, 1, 0, 1, 1, 0, 0, 0),
  (1, 1, 0, 1, 1, 1, 0, 0, 0),
  (1, 0, 0, 1, 0, 0, 1, 0, 0),
  (0, 1, 1, 1, 0, 0, 1, 0, 0),
  (0, 0, 1, 0, 1, 0, 1, 0, 0),
  (1, 1, 1, 0, 1, 0, 1, 0, 0),
  (1, 0, 0, 1, 1, 0, 1, 0, 0),
  (1, 0, 1, 1, 1, 0, 1, 0, 0),
  (1, 1, 1, 0, 0, 1, 1, 0, 0),
  (0, 1, 0, 1, 1, 1, 1, 0, 0),
  (0, 1, 1, 0, 1, 1, 0, 1, 0)}}

Two vertices are distinguished: the sum of labels of their incident edges is $16$, as opposed to $17$ for all other vertices.

They are $[A_0]$ and $[\omega-A_0]$:

In [16]:
sum_edge_labels[16] == {representative(A.column(0)), representative(omega-A.column(0))}

True

In [17]:
A0_repr = representative(A.column(0))

In [18]:
autH = H.automorphism_group()

In [19]:
stabA0 = autH.stabilizer(A0_repr)

Compute the orbits of the action of $\operatorname{Stab}([A_0]) < \operatorname{Aut}(\mathcal H)$
on the set of neighbors of $[A_0]$:

In [20]:
orbits = sorted({tuple(sorted(stabA0.orbit(v))) for v in H[A0_repr]})
for orb in orbits:
    print(orb)

((0, 1, 0, 0, 0, 0, 0, 0, 0), (0, 1, 1, 0, 0, 0, 0, 0, 0), (0, 0, 1, 1, 1, 0, 0, 0, 0), (0, 1, 1, 0, 1, 1, 0, 1, 0))
((1, 1, 0, 1, 0, 0, 0, 0, 0), (1, 1, 1, 1, 0, 0, 0, 0, 0), (1, 1, 0, 1, 1, 1, 0, 0, 0), (1, 1, 1, 0, 1, 0, 1, 0, 0))


Let us show that the action of $\operatorname{Stab}([A_0]) < \operatorname{Aut}(\mathcal H)$ on the neighbors of $[A_0]$ has two orbits: 
- $\{[A_0 + A_1], [A_0 + A_2],[A_0 + A_4],[A_0 + A_8]\}$;
- $\{[A_0 + A_3], [A_0 + A_5],[A_0 + A_6],[A_0 + A_7]\}$.

In [21]:
for j in range(1,9):
    rj = representative(A.column(0) + A.column(j))
    if rj in orbits[0]:
        idx = 0
    elif rj in orbits[1]:
        idx = 1
    else:
        raise ValueError()
    print(j, rj, f"in orbits[{idx}]")

1 (0, 1, 0, 0, 0, 0, 0, 0, 0) in orbits[0]
2 (0, 1, 1, 0, 0, 0, 0, 0, 0) in orbits[0]
3 (1, 1, 1, 1, 0, 0, 0, 0, 0) in orbits[1]
4 (0, 0, 1, 1, 1, 0, 0, 0, 0) in orbits[0]
5 (1, 1, 0, 1, 1, 1, 0, 0, 0) in orbits[1]
6 (1, 1, 0, 1, 0, 0, 0, 0, 0) in orbits[1]
7 (1, 1, 1, 0, 1, 0, 1, 0, 0) in orbits[1]
8 (0, 1, 1, 0, 1, 1, 0, 1, 0) in orbits[0]


Now consider another graph $\mathcal H'$: its vertices are the $30$ copies of $Y_i$ in our manifold, and two copies of $Y_i$ sharing $k$ facets are joined by an edge labeled $k/16$ (or by no edge if $k=0$). By construction, there is a label-preserving isomorphism between $\mathcal H'$ and $\mathcal H$.

Let $y_1$ be a copy of $Y_i$ adjacent via $16$ facets to (exactly) one of the two quotiented copies $y_0$ of $Y_i$; it corresponds to $[A_0]$ or $[\omega-A_0]$ in $\mathcal H$. We can assume the former, as there is an automorphism of $\mathcal H$ swapping $[A_0]$ with $[\omega-A_0]$:

In [22]:
set(autH.orbit(A0_repr)) == {representative(A.column(0)), representative(omega-A.column(0))}

True

If $y_2$ is a neighbor of $y_1$ in $\mathcal H'$, then $y_1$ and $y_2$ share $32 = 2\cdot 16$ facets:

In [23]:
[c for a,b,c in H.edges(A0_repr)]

[2, 2, 2, 2, 2, 2, 2, 2]

These shared facets induce reflection isometries $y_1 \leftrightarrow y_2$. The composition of two such reflections is an isometry $\xi$ of $y_1$; if we choose two facets that had different colors in $\widehat{M}_{i,I}^\pm$, then $\xi$ generates the order-$17$ isometry group used in the definition of the coloring. In fact, if $\phi$ is the generator used in the construction, then $\xi = \phi^k$, where:
- $k$ is square modulo $17$ if $y_2$ corresponds to a vertex in $\{[A_0 + A_1], [A_0 + A_2],[A_0 + A_4],[A_0 + A_8]\}$;
- $k$ is non-square modulo $17$ if $y_2$ corresponds to a vertex in $\{[A_0 + A_3], [A_0 + A_5],[A_0 + A_6],[A_0 + A_7]\}$.

Since we are able to distinguish the two sets, even up to $\operatorname{Stab}([A_0])$, we may choose $y_2$ in the first set, and recover the type of $\phi$, which is the same as the type of $\xi = \phi^k$.

Lastly, up to isometry, we can assume $\langle \phi \rangle = \langle \psi \rangle$ and recover the labeling of $\mathcal G$ up to an isometry of $Y_i$ normalizing $\langle \psi \rangle$, i.e. up to $\Lambda_i$. \
The independent set $I$ can be read off from the facets of $y_1$ adjacent to $y_0$.